In [1]:
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import StatevectorEstimator as Estimator, StatevectorSampler as Sampler

#Pyscf import
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
from qiskit.circuit.library import TwoLocal
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
# General imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time

# Import necessary libraries
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
from openfermionpyscf import generate_molecular_hamiltonian
from openfermion.linalg import get_sparse_operator
import py3Dmol  # For molecule visualization
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms.minimum_eigensolvers import AdaptVQE, VQE

# Notebook setup
from IPython.display import display, Markdown

In [2]:
# Display step function
def display_step(message):
    print(message)

display_step("Step 1: Define the H₂ molecule geometry and basis set")
geometry = [('Ba', (0, 0, 0)), ('H', (0, 0, 0.74))]  # Bond distance ~0.74 Å
basis = 'sto-3g'  # Minimal basis set
multiplicity = 1  # Singlet state (closed shell)
charge = 0  # Neutral H₂ molecule

display_step(f"Defined geometry: {geometry}\nUsing basis: {basis}")

# Define distances for the geometry
# For H2, we will vary the distance between hydrogen atoms to see the energy profile
distances  = [x * 0.05 for x in range(6, 60)]

energies = []
hf_energies = []
estimator = Estimator()


print(f'Processing step __', end='')
for i, d in enumerate(distances):
    print('\b\b{:2d}'.format(i), end='', flush=True)
    # Update geometry with current distance
    geometry = [('Ba', (0, 0, 0)), ('H', (0, 0, d))]
    driver = PySCFDriver(atom='; '.join([f'{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}' for atom in geometry]), basis=basis, charge=charge, spin=multiplicity - 1)
    molecule = driver.run()

    # Freeze core orbitals if applicable
    transformer = FreezeCoreTransformer()
    molecule = transformer.transform(molecule)

    # Generate the Hamiltonian
    hamiltonian = molecule.hamiltonian.second_q_op()

    # Use ParityMapper and get tapered mapper
    mapper = ParityMapper()
    tapered_mapper = molecule.get_tapered_mapper(mapper)

    # Set up the ansatz and VQE
    optimizer = SLSQP(maxiter=10000, ftol=1e-9)
    ansatz = UCCSD(
        molecule.num_spatial_orbitals,
        molecule.num_particles,
        tapered_mapper,
        initial_state=HartreeFock(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            tapered_mapper,
        ),
    )
    vqe = VQE(estimator, ansatz, optimizer)
    adapt_vqe = AdaptVQE(vqe)
    adapt_vqe.initial_point = [0] * ansatz.num_parameters
    algo = GroundStateEigensolver(tapered_mapper, adapt_vqe)
    result = algo.solve(molecule)

    # Store energies
    energies.append(result.total_energies[0])
    hf_energies.append(result.hartree_fock_energy)

print(' --- complete')


# Find the lowest VQE, Hartree-Fock, and FCI energies
lowest_vqe_energy = min(energies)
lowest_hf_energy = min(hf_energies)

# Find the corresponding distances
lowest_vqe_distance = distances[energies.index(lowest_vqe_energy)]
lowest_hf_distance = distances[hf_energies.index(lowest_hf_energy)]

# Print the lowest energies and corresponding bond distances
print(f"Lowest VQE Energy: {lowest_vqe_energy:.8f} Hartree at bond distance {lowest_vqe_distance:.2f} Å")
print(f"Lowest Hartree-Fock Energy: {lowest_hf_energy:.8f} Hartree at bond distance {lowest_hf_distance:.2f} Å")

# Plotting the energy profile
fig, ax = plt.subplots()
ax.plot(distances, energies, label=" ADAPT VQE Energy")
ax.plot(distances, hf_energies, label="Hartree-Fock Energy", linestyle='--')
ax.set_xlabel("Bond Distance (Å)")
ax.set_ylabel("Energy (Hartree)")
plt.title('Potential Energy Surface of BeH')

ax.legend()
plt.show()

Step 1: Define the H₂ molecule geometry and basis set
Defined geometry: [('Ba', (0, 0, 0)), ('H', (0, 0, 0.74))]
Using basis: sto-3g
Processing step  0

/tmp/ipykernel_243890/1507649426.py:19: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


QiskitNatureError: 'Failed to build the PySCF Molecule object.'

In [3]:
# Display step function
def display_step(message):
    print(message)

def calculate_ground_state_energy():
    display_step("Calculating ground state energy for bond distance 0.74 Å")
    geometry = [('H', (0, 0, 0)), ('H', (0, 0, 0.74))]
    driver = PySCFDriver(atom='; '.join([f'{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}' for atom in geometry]), basis=basis, charge=charge, spin=multiplicity - 1)
    molecule = driver.run()


    # Freeze core orbitals if applicable
    transformer = FreezeCoreTransformer()
    molecule = transformer.transform(molecule)

    # Generate the Hamiltonian
    hamiltonian = molecule.hamiltonian.second_q_op()

    # Use ParityMapper and get tapered mapper
    mapper = ParityMapper()
    tapered_mapper = molecule.get_tapered_mapper(mapper)
    
    # Set up the ansatz and VQE
    optimizer = SLSQP(maxiter=10000, ftol=1e-9)
    ansatz = UCCSD(
        molecule.num_spatial_orbitals,
        molecule.num_particles,
        tapered_mapper,
        initial_state=HartreeFock(
            molecule.num_spatial_orbitals,
            molecule.num_particles,
            tapered_mapper,
        ),
    )
    vqe = VQE(estimator, ansatz, optimizer)
    vqe.initial_point = [0] * ansatz.num_parameters
    algo = GroundStateEigensolver(tapered_mapper, vqe)
    result = algo.solve(molecule)

    fci_energy = molecule.nuclear_repulsion_energy + result.total_energies[0]
    return result.total_energies[0], result.hartree_fock_energy, fci_energy

display_step("Step 1: Define the H₂ molecule geometry and basis set")
geometry = [('H', (0, 0, 0)), ('H', (0, 0, 0.74))]  # Bond distance ~0.74 Å
basis = 'sto-3g'  # Minimal basis set
multiplicity = 1  # Singlet state (closed shell)
charge = 0  # Neutral H₂ molecule

display_step(f"Defined geometry: {geometry}\nUsing basis: {basis}")


energies = []
hf_energies = []
estimator = Estimator()


vqe_energy, hf_energy, fci_energy = calculate_ground_state_energy()
energies.append(vqe_energy)
hf_energies.append(hf_energy)

print(' --- complete')

# Print the calculated energies
print(f"VQE Energy: {vqe_energy:.8f} Hartree")
print(f"Hartree-Fock Energy: {hf_energy:.8f} Hartree")


Step 1: Define the H₂ molecule geometry and basis set
Defined geometry: [('H', (0, 0, 0)), ('H', (0, 0, 0.74))]
Using basis: sto-3g
Calculating ground state energy for bond distance 0.74 Å
 --- complete
VQE Energy: -1.13728383 Hartree
Hartree-Fock Energy: -1.11675931 Hartree


/tmp/ipykernel_421917/1938643308.py:54: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
